In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator

from indra_db_lite.api import get_entrez_pmids_for_hgnc
from indra_db_lite.api import get_entrez_pmids_for_uniprot
from indra_db_lite.api import get_plaintexts_for_text_ref_ids
from indra_db_lite.api import get_text_ref_ids_for_agent_text
from indra_db_lite.api import get_text_ref_ids_for_pmids


from adeft_indra.grounding import AdeftGrounder
from adeft_indra.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename

In [2]:
def get_text_ref_ids_for_entity(ns, id_):
    if ns == 'HGNC':
        pmids = get_entrez_pmids_for_hgnc(id_)
    elif ns == 'UP':
        pmids = get_entrez_pmids_for_uniprot(id_)
    return list(get_text_ref_ids_for_pmids(pmids).values())

In [3]:
adeft_grounder = AdeftGrounder()

In [5]:
shortforms = ['CAK', 'CAKs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../../', 'results', model_name))

In [6]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    text_ref_ids = get_text_ref_ids_for_agent_text(shortform)
    content = get_plaintexts_for_text_ref_ids(text_ref_ids, contains=shortforms)
    text_dict = content.flatten()
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [7]:
list(zip(longforms, counts))

[('cdk activating kinase', 221),
 ('cyclin dependent kinase activating kinase', 18),
 ('cyclin activating kinase', 10),
 ('activating kinase complex', 5),
 ('cdk activating kinases', 4)]

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'activating kinase complex': 'FPLX:CDK_activating_kinase_complex',
  'cdk activating kinase': 'FPLX:CDK_activating_kinase_complex',
  'cdk activating kinases': 'FPLX:CDK_activating_kinase_complex',
  'cyclin activating kinase': 'FPLX:CDK_activating_kinase_complex',
  'cyclin dependent kinase activating kinase': 'FPLX:CDK_activating_kinase_complex'},
 {'FPLX:CDK_activating_kinase_complex': 'CDK_activating_kinase_complex'},
 ['FPLX:CDK_activating_kinase_complex']]

In [11]:
[{'activating kinase complex': 'FPLX:CDK_activating_kinase_complex',
  'cdk activating kinase': 'FPLX:CDK_activating_kinase_complex',
  'cdk activating kinases': 'FPLX:CDK_activating_kinase_complex',
  'cyclin activating kinase': 'FPLX:CDK_activating_kinase_complex',
  'cyclin dependent kinase activating kinase': 'FPLX:CDK_activating_kinase_complex'},
 {'FPLX:CDK_activating_kinase_complex': 'CDK_activating_kinase_complex'},
 ['FPLX:CDK_activating_kinase_complex']]

[{'activating kinase complex': 'FPLX:CDK_activating_kinase_complex',
  'cdk activating kinase': 'FPLX:CDK_activating_kinase_complex',
  'cdk activating kinases': 'FPLX:CDK_activating_kinase_complex',
  'cyclin activating kinase': 'FPLX:CDK_activating_kinase_complex',
  'cyclin dependent kinase activating kinase': 'FPLX:CDK_activating_kinase_complex'},
 {'FPLX:CDK_activating_kinase_complex': 'CDK_activating_kinase_complex'},
 ['FPLX:CDK_activating_kinase_complex']]

In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [62]:
additional_entities = {
    'HGNC:2730': ['DDR1', ['DDR1', 'CAK', 'cell adhesion kinase']],
    'HGNC:2536': ['CTSK', ['CAK', 'cathepsin']],
    
}

In [63]:
unambiguous_agent_texts = {
    'HGNC:2730': ['DDR1', ['cell adhesion kinase']],
    'HGNC:2536': ['CTSK', ['cathepsin k']]
}

In [64]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts(
    (text, text_ref_id) for text_ref_id, text in all_texts.items()
)
agent_text_text_ref_id_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_text_ref_id_map[label].append(id_)

entity_text_ref_id_map = {
    entity: set(
        get_text_ref_ids_for_entity(*entity.split(':', maxsplit=1))
    )
    for entity in additional_entities
}

In [65]:
intersection1 = []
for entity1, trids1 in entity_text_ref_id_map.items():
    for entity2, trids2 in entity_text_ref_id_map.items():
        intersection1.append((entity1, entity2, len(trids1 & trids2)))

In [66]:
intersection2 = []
for entity1, trids1 in agent_text_text_ref_id_map.items():
    for entity2, pmids2 in entity_text_ref_id_map.items():
        intersection2.append((entity1, entity2, len(set(trids1) & trids2)))

In [67]:
intersection1

[('HGNC:2730', 'HGNC:2730', 191),
 ('HGNC:2730', 'HGNC:2536', 6),
 ('HGNC:2536', 'HGNC:2730', 6),
 ('HGNC:2536', 'HGNC:2536', 186)]

In [68]:
intersection2

[('FPLX:CDK_activating_kinase_complex', 'HGNC:2730', 0),
 ('FPLX:CDK_activating_kinase_complex', 'HGNC:2536', 0)]

In [69]:
all_used_trids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_trids = set()
    for agent_text in agent_texts[1]:
        trids = set(get_text_ref_ids_for_agent_text(agent_text))
        new_trids = list(trids - all_texts.keys() - used_trids)
        content = get_plaintexts_for_text_ref_ids(new_trids, contains=agent_texts[1])
        text_dict = content.flatten()
        corpus.extend(
            [
                (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
            ]
        )
        used_trids.update(new_trids)
    all_used_trids.update(used_trids)
        
for entity, trids in entity_text_ref_id_map.items():
    new_trids = list(set(trids) - all_texts.keys() - all_used_trids)
    _, contains = additional_entities[entity]
    content = get_plaintexts_for_text_ref_ids(new_trids, contains=contains)
    text_dict = content.flatten()
    corpus.extend(
        [
            (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
        ]
    )

In [70]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [71]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2021-10-05 17:05:41] /adeft/Py/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2021-10-05 17:05:49] /adeft/Py/adeft/adeft/modeling/classify.py - Best f1 score of 0.9918573490002063 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [72]:
classifier.stats

{'label_distribution': {'FPLX:CDK_activating_kinase_complex': 212,
  'HGNC:2730': 127,
  'HGNC:2536': 152},
 'f1': {'mean': 0.991857, 'std': 0.004072},
 'precision': {'mean': 0.991857, 'std': 0.004072},
 'recall': {'mean': 0.991857, 'std': 0.004072},
 'FPLX:CDK_activating_kinase_complex': {'f1': {'mean': 0.990586,
   'std': 0.00471},
  'pr': {'mean': 0.995238, 'std': 0.009524},
  'rc': {'mean': 0.986152, 'std': 0.011308}},
 'HGNC:2536': {'f1': {'mean': 0.996721, 'std': 0.006557},
  'pr': {'mean': 0.993548, 'std': 0.012903},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:2730': {'f1': {'mean': 0.988223, 'std': 0.009628},
  'pr': {'mean': 0.984308, 'std': 0.019225},
  'rc': {'mean': 0.992593, 'std': 0.014815}}}

In [73]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [74]:
disamb.dump(model_name, results_path)

In [80]:
print(disamb.info())

Disambiguation model for CAK, and CAKs

Produces the disambiguations:
	CDK_activating_kinase_complex*	FPLX:CDK_activating_kinase_complex
	CTSK*	HGNC:2536
	DDR1*	HGNC:2730

Class level metrics:
--------------------
Grounding                    	Count	F1     
CDK_activating_kinase_complex*	212	0.99059
                         CTSK*	152	0.99672
                         DDR1*	127	0.98822

Global Metrics:
-----------------
	F1 score:	0.99186
	Precision:	0.99186
	Recall:		0.99186

* Positive labels
See Docstring for explanation



In [25]:
model_to_s3(disamb)

In [26]:
from adeft.disambiguate import load_disambiguator

In [27]:
disamb = load_disambiguator("BAL")

In [28]:
disamb

In [29]:
print(_28.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage	MESH:D018893
	CEL*	HGNC:1848
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase*	MESH:C059416
	betaine aldehyde*	CHEBI:CHEBI:15710
	dimercaprol*	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage	1259	 0.9929
                   CEL*	  36	    1.0
     Liver, Artificial	  18	0.83619
            Ungrounded	  17	   0.65
           dimercaprol*	   8	    0.4
    benzaldehyde lyase*	   3	    0.2
      betaine aldehyde*	   2	    0.2

Global Metrics:
-----------------
	F1 score:	0.90773
	Precision:	1.0
	Recall:		0.83293

* Positive labels
See Docstring for explanation



/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 

In [44]:
labeled_trids = set(row[-1] for row in corpus)

In [46]:
unlabled = [text for trid, text in all_texts.items() if trid not in labeled_trids]

In [81]:
X = disamb.disambiguate(unlabled)

In [82]:
d = [(text, pred) for pred, text in zip(X, unlabled) if pred[0].startswith("HGNC")]

In [87]:
d[1][0]

'The first identified regulated phosphorylation process involving CAS proteins was in the context of cellular attachment to extracellular matrix, activation of integrins and subsequent spreading [7, 11, 12, 14, 26, 33, 46–48, 76–78]. As shown in Figure 2, soon after cell attachment, integrin signaling activates FAK (or depending on cell type, the FAK paralog RAFTK/CAK-β/Pyk2) [79]. FAK binds directly to CAS proteins, based in part on interactions between the CAS SH3 domain and FAK polyproline sequences. FAK phosphorylates CAS proteins (BCAR1, NEDD9, EFS) directly on the C-terminal binding site YDYVHL [77]. This creates a high affinity site for binding by the SH2 domain of SRC family kinases (including SRC, LYN, FYN, and others), contributing to the recruitment of these kinases. SRC kinases then rapidly phosphorylate multiple tyrosine phosphorylation motifs within the CAS SD, creating binding sites for proteins that promote cell spreading, including most notably Crk and Crk-L.\nExtensiv